# CS6109 - Compiler Design Project

**Project Topic: Social Recommendations with multiple influence from Direct User Interactions**

Done by:

    Vivek Ramkumar, 2018103082
    Kariketi Tharun Reddy, 201803034
    G. R. Srikanth, 2018103603
                                                    
    Third Years studying B.E, Computer Science, at College of Engineering, Guindy
    Chennai, Tamil Nadu, India

# Importing Standard Libraries:

In [ ]:
pip install advertools

In [ ]:
import numpy as np
import pandas as pd 
from collections import OrderedDict
import gc
import random

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Obtaining Data:

We are using three datasets of products, from which recommendations will be made.

1. Groceries Dataset
2. Flipkart Product Database
3. Consumer Reviews of Amazon Products

While extracting data from the respective databases, we need to ensure there are no duplicate items or NaN values.

So, we use OrderedDict.fromkeys() to remove all duplicate items. We then check whether any NaN values exist 
in the list of products. If there are NaN values, we promptly exclude them.

Finally, we delete the database as it occupies a lot of space in the RAM.
gc.collect() is a garbage collection function used to clear up any extra leftovers from the deletion.

This process is repeated for all the three databases.

In [ ]:
groceries_db = pd.read_csv('../input/groceries/groceries - groceries.csv')

new_groceries_db = groceries_db.drop(columns = ['Item(s)'])

temp = []

for i in range(len(new_groceries_db.columns)):
    for x in new_groceries_db.iloc[:, i]:
        temp.append(x)
            
groceries = [] 

groceries = list(OrderedDict.fromkeys(temp))

groceries = [item for item in groceries if str(item) != 'nan']

del groceries_db, new_groceries_db
gc.collect()

In [ ]:
flipkart_db = pd.read_csv('../input/flipkart-products/flipkart_com-ecommerce_sample.csv')

temp = flipkart_db['product_name']

flipkart_items = [] 

flipkart_items = list(OrderedDict.fromkeys(temp))

flipkart_items = [item for item in flipkart_items if str(item) != 'nan']

del flipkart_db
gc.collect()

In [ ]:
def read_amzn_prod_dbs(file_path):
    
    temp_db = pd.read_csv(file_path)

    temp = temp_db['name']
    
    return temp
    
def append_list_to_list(orig_list, new_list):
    
    for x in orig_list:
        new_list.append(x)

temp_1 = read_amzn_prod_dbs('../input/consumer-reviews-of-amazon-products/1429_1.csv')
temp_2 = read_amzn_prod_dbs('../input/consumer-reviews-of-amazon-products/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv')
temp_3 = read_amzn_prod_dbs('../input/consumer-reviews-of-amazon-products/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

temp = []

append_list_to_list(temp_1, temp)
append_list_to_list(temp_2, temp)
append_list_to_list(temp_3, temp)

amazon_items = [] 

amazon_items = list(OrderedDict.fromkeys(temp))

amazon_items = [item for item in amazon_items if str(item) != 'nan']

We combine all the obtained lists of products into a singular mega-list.
    
This list will be referred to for recommending products to the users.

In [ ]:
complete_list = []

append_list_to_list(flipkart_items, complete_list)
append_list_to_list(groceries, complete_list)
append_list_to_list(amazon_items, complete_list)

In [ ]:
print("The total number of products available to recommend :", len(complete_list))

**The mega-list contains over 12,000 product names. This is a sizeable amount, enough to get different recommendations on every run.**

# Lexical Analysis:

The main focus of lexical analysis is to obtain tokens from the given text input.

We need to differentiate between entities (users, organizations, religion, etc.) and interests (garments, tech, etc.).

The technique we are going to employ is called POS tagging. (POS -> Part of Speech)
    
POS tagging algorithms allocate tags to each token, such as "NOUN", "VERB", "ADJ", etc., based on the similarity to their respective tagged categories.

We will be using the Spacy library, which contains pre-trained NLP models and a huge dictionary of words.

The input is parsed sentence by sentence, from which nouns, verbs, users, and user types are obtained. The respective information is summarized in a pandas dataframe. You will see it in the 'Input and Output' section.

In [ ]:
import nltk
import spacy

from spacy import displacy

from nltk.corpus import stopwords

nltk.download('stopwords')

from nltk.tokenize import word_tokenize

In [ ]:
def tokenizer(text, nouns, verbs, users, user_types):
    
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    
    for token in doc:
        if token.pos_ == 'NOUN':
            nouns.append(str(token))

        if token.pos_ == 'VERB':
            verbs.append(str(token))

    for entity in doc.ents:
        users.append(entity.text)
        user_types.append(entity.label_)
    
    nouns = [word for word in nouns if not word in stopwords.words()]

Keep in mind the input is made to understand short conversations - especially one-liners.

Basically, we separate the text blob by the '\n', or newline symbol.
This is done by using the splitlines() function.

If any respective dialogue consists of two lines, the parser will be confused and allocate an extra user to the extra line.

This is done with consideration to social messaging applications, as users frequently communicate with short dialogues.

In [ ]:
def get_tokens(text):
    
    conversation = text.splitlines()

    token_summary = pd.DataFrame({'Users': [], 'Nouns': [], 'Verbs': [], 'User_Types': []})

    for dialogue in conversation:
    
        nouns = []
        verbs = []
        users = []
        user_types = []

        tokenizer(dialogue, nouns, verbs, users, user_types)
    
        nouns = list(OrderedDict.fromkeys(nouns))
        verbs = list(OrderedDict.fromkeys(verbs))
    
        token_summary = token_summary.append({'Users': users, 'Nouns': nouns, 'Verbs': verbs, 'User_Types': user_types}, ignore_index=True)
    
    return conversation, token_summary

# Syntax Analysis:

The main point of understanding the syntax of the sentence is to check the order by which tokens are parsed.

For example, seeing a "not" before a verb, say "enjoying", can imply a negative sentiment.

Sentiment analysis is an important tool in recommendation - it helps recommend products to those who want it.

People who express negative sentiment towards a product will not recieve recommendations on it.
    
Instead, they will get recommendations based on what their friends like.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def get_sentiments(conversation):

    sentiments = pd.DataFrame({'Dialogue': [], 'Compound': [], 'Negative': [], 'Neutral': [], 'Positive': []})

    sid = SentimentIntensityAnalyzer()

    for dialogue in conversation:
    
        ss = sid.polarity_scores(dialogue)
    
        sentiments = sentiments.append({'Dialogue': dialogue, 'Compound': ss['compound'], 'Negative': ss['neg'], 'Neutral': ss['neu'], 'Positive': ss['pos']}, ignore_index = True)
        
    return sentiments

The NTLK Vader Sentiment Analyzer is an excellent tool for quickly analyzing the sentiment of a sentence.

We will obtain the polarity scores in each sentiment category, i.e "Positive", "Negative", "Neutral", and "Compound", for each user.

A summary of the sentiments expressed by each user will be shown in the 'Input and Output' section.

In [ ]:
def display_dependency_tree(conversation):

    nlp = spacy.load("en_core_web_sm")

    for dialogue in conversation:
    
        doc = nlp(dialogue)

        displacy.render(doc, style="dep")

The Spacy library consists of one more feature - the ability to show dependency trees.

Dependency trees are useful in understanding the parsing process, in pursuit of finding the sentiments.

You can see the respective dependency trees for each user in the "Input and Output" section.

# Recommendation(Semantic):

This is the recommendation section.

We will check the list of nouns from our summary of tokens to see what the interests of the users are.

If the particular noun is present in any product name, we will store that product name in a new list.

This new list is considered as our initial set of recommendations.

In [ ]:
def create_init_rec(token_summary):

    rec_1 = []

    for i in range(len(token_summary['Nouns'])):
    
        sublist = []
    
        for x in token_summary['Nouns'][i]:
        
            for y in complete_list:
            
                if x.lower() in y.lower():
                    sublist.append(y)
        
        rec_1.append(sublist)
        
    return rec_1

Now, our initial recommendations are quite a large number.

We will pick five products at random, to recommend to each user.

Of course, we do not want to get the same products again in our random selection.

This is easily achieved by using random.sample(list, k), where k is the number of items selected. This function also ensures that there is no repetition in the selection of items.

In [ ]:
def get_actual_rec(rec_1):

    recommendations = []

    for i in range(len(rec_1)):
    
        sublist = random.sample(rec_1[i], 5)
    
        recommendations.append(sublist)
        
    return recommendations

In [ ]:
def mixed_rec(rec, index):
    
    temp = []
    
    for i in range(len(rec)):
        
        if(i==index):
            if(i == len(rec)-1):
                temp.append(random.choice(rec[i-1]))
            else:
                temp.append(random.choice(rec[i+1]))
        else:
            temp.append(random.choice(rec[i]))
        
    return temp

As stated before, the sentiment of the user is considered as a factor in the recommendation process.

The compounded polarity score is considered here.

If a user's sentiment is negative, then we pick a random recommendation based on the interests of that user's friends.

Otherwise, if it is positive or neutral, we can directly recommend based on their individual interest.

In [ ]:
def print_rec():

    print("Recommendations are: \n")

    i = 0

    for u in token_summary['Users']:
            
        for x in u:
            
            if(sentiments['Compound'][i] > 0):     
                print(x, '->', recommendations[i])
            elif(sentiments['Compound'][i] < 0):
                print(x, '->', mixed_rec(recommendations, i))
            else:
                print(x, '->', recommendations[i])
        
            print()
            i = i + 1 

# Input and Output

**This is the input text.**

**Every time you want to get some new recommendations, tweak this and run the cells below it.**

In [ ]:
text = """ Rahul : I like to play on my tablet.
 Gokul : My dog does not enjoy using dog shampoo.
 Ankita : I love to wear a kurta.
 Badrinath : I frequently wear a shirt.
 Vijay : I am searching for a nice watch. 
 3 : I like to play cricket.
 John: This shirt is my favourite.
 Adam: Shall we buy a new watch?
 Oliver: I got a new smart phone.
 Vikram: I really like this mouse.
 Ramaswamy: I am planning to buy a watch.
 Sabu : Do you have the charger?
 Seetha: I am eager to see my new saree.
 Meenakshi: My parrot sat on the sofa.
 Parameshwar : I am waiting to get the laptop.
 Tom: These people need a good refrigerator.
 Sally: My mother wants a hat for the hot summers.
 Krishna: That is the song for keyboard.
 Mary: The cake is ready in the oven. 
 Nicholas: I really do not like this chair. 
 Sania: I love playing tennis. 
 Sigmund: Do you have a discount on pet food?
 Rajan: It would be nice to get a new coffee mug.
 Regina: This new fan runs so smoothly!
 Nakatomi: One must always have a wooden table.
 Deepak: I love my dancing shoes.
 Watson: You should shine one side of the ball to spin.
 Maya: It's so chilly, I need a scarf.
 Shigeru: Back to the drawing board.
 Lara: Look at the cute brooch I found!
 Sawyer: This bike horn has been polished very nicely. """

In [ ]:
conversation, token_summary = get_tokens(text)

In [ ]:
token_summary

In [ ]:
sentiments = get_sentiments(conversation)

In [ ]:
sentiments

Shown below are the dependency trees for each sentence.

The arrows infer dependencies between words. If an arrow points from 'a' to 'b', then it can be understood that 'b' depends on 'a'.

This is useful in seeing how the tokens are tagged, and how the sentence is syntactically analyzed.

In [ ]:
display_dependency_tree(conversation)

This is the final recommendation.

You can see the user's names followed by a list of recommendations based on their interests.
    
Keep in mind that for the most part the recommendations are accurate, but you may chance upon some outliers, due to the random nature of product selection.

In [ ]:
rec_1 = create_init_rec(token_summary)
rec_1


In [ ]:
recommendations = get_actual_rec(rec_1)
print_rec()

**You have reached the end of this notebook. Thank you for reading!**